In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv("Algerian_forest_fires_clean_dataset.csv",index_col="Unnamed: 0")

In [4]:
df.head()

day  month  year  Temperature  RH  Ws  Rain  FFMC  DMC    DC  ISI  BUI  \
0    1      6  2012           29  57  18   0.0  65.7  3.4   7.6  1.3  3.4   
1    2      6  2012           29  61  13   1.3  64.4  4.1   7.6  1.0  3.9   
2    3      6  2012           26  82  22  13.1  47.1  2.5   7.1  0.3  2.7   
3    4      6  2012           25  89  13   2.5  28.6  1.3   6.9  0.0  1.7   
4    5      6  2012           27  77  16   0.0  64.8  3.0  14.2  1.2  3.9   

   FWI      Classes  Region  
0  0.5  not fire          0  
1  0.4  not fire          0  
2  0.1  not fire          0  
3  0.0  not fire          0  
4  0.5  not fire          0

In [5]:
df.columns

Index(['day', 'month', 'year', 'Temperature', 'RH', 'Ws', 'Rain', 'FFMC',
       'DMC', 'DC', 'ISI', 'BUI', 'FWI', 'Classes', 'Region'],
      dtype='object')

In [6]:
df.drop(['day','month','year'],axis=1,inplace=True)

In [7]:
df.head()

Temperature  RH  Ws  Rain  FFMC  DMC    DC  ISI  BUI  FWI      Classes  \
0           29  57  18   0.0  65.7  3.4   7.6  1.3  3.4  0.5  not fire      
1           29  61  13   1.3  64.4  4.1   7.6  1.0  3.9  0.4  not fire      
2           26  82  22  13.1  47.1  2.5   7.1  0.3  2.7  0.1  not fire      
3           25  89  13   2.5  28.6  1.3   6.9  0.0  1.7  0.0  not fire      
4           27  77  16   0.0  64.8  3.0  14.2  1.2  3.9  0.5  not fire      

   Region  
0       0  
1       0  
2       0  
3       0  
4       0

In [8]:
df['Classes']=np.where(df['Classes'].str.contains("not fire"),0,1)

In [9]:
df['Classes'].value_counts()

Classes
1    137
0    106
Name: count, dtype: int64

In [10]:
X=df.drop(['FWI'],axis=1)
y=df['FWI']

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [12]:
#feature selection
x_train.corr()

Temperature        RH        Ws      Rain      FFMC       DMC  \
Temperature     1.000000 -0.656095 -0.305977 -0.317512  0.694768  0.498173   
RH             -0.656095  1.000000  0.225736  0.241656 -0.653023 -0.414601   
Ws             -0.305977  0.225736  1.000000  0.251932 -0.190076  0.000379   
Rain           -0.317512  0.241656  0.251932  1.000000 -0.545491 -0.289754   
FFMC            0.694768 -0.653023 -0.190076 -0.545491  1.000000  0.620807   
DMC             0.498173 -0.414601  0.000379 -0.289754  0.620807  1.000000   
DC              0.390684 -0.236078  0.096576 -0.302341  0.524101  0.868647   
ISI             0.629848 -0.717804 -0.023558 -0.345707  0.750799  0.685656   
BUI             0.473609 -0.362317  0.035633 -0.300964  0.607210  0.983175   
Classes         0.542141 -0.456876 -0.082570 -0.369357  0.781259  0.617273   
Region          0.254549 -0.394665 -0.199969 -0.059022  0.249514  0.212582   

                   DC       ISI       BUI   Classes    Region  
Temperature  0.390684  0.629848  0.473609  0.542141  0.254549  
RH          -0.236078 -0.717804 -0.362317 -0.456876 -0.394665  
Ws           0.096576 -0.023558  0.035633 -0.082570 -0.199969  
Rain        -0.302341 -0.345707 -0.300964 -0.369357 -0.059022  
FFMC         0.524101  0.750799  0.607210  0.781259  0.249514  
DMC          0.868647  0.685656  0.983175  0.617273  0.212582  
DC           1.000000  0.513701  0.942414  0.543581 -0.060838  
ISI          0.513701  1.000000  0.643818  0.742977  0.296441  
BUI          0.942414  0.643818  1.000000  0.612239  0.114897  
Classes      0.543581  0.742977  0.612239  1.000000  0.188837  
Region      -0.060838  0.296441  0.114897  0.188837  1.000000

In [13]:
plt.figure(figsize=(20,15))
sns.heatmap(x_train.corr(),annot=True)
plt.show()

In [14]:
def correlation(dataset,threshold):
    features=set()
    col_corr=dataset.corr()
    for i in range(len(col_corr)):
        for j in range(i):
            if abs(col_corr.iloc[i, j]) > threshold: 
                colname = col_corr.columns[i]
                features.add(colname)
    return features
            

In [15]:
correlation(x_train,0.85)

{'BUI', 'DC'}

In [16]:
x_train.drop(correlation(x_train,0.85),axis=1,inplace=True)
x_test.drop(['BUI', 'DC'],axis=1,inplace=True)

In [17]:
x_test.head()

Temperature  RH  Ws  Rain  FFMC   DMC  ISI  Classes  Region
24            31  64  15   0.0  86.7  14.2  5.7        1       0
6             33  54  13   0.0  88.2   9.9  6.4        1       0
152           28  58  18   2.2  63.7   3.2  1.2        0       1
232           34  58  13   0.2  79.5  18.7  2.1        0       1
238           30  65  14   0.0  85.4  16.0  4.5        1       1

In [18]:
x_train.head()

Temperature  RH  Ws  Rain  FFMC   DMC   ISI  Classes  Region
97            29  74  19   0.1  75.8   3.6   2.1        0       0
141           31  72  14   0.2  60.2   3.8   0.8        0       1
192           40  31  15   0.0  94.2  22.5  16.6        1       1
68            32  60  18   0.3  77.1  11.3   2.2        0       0
119           32  47  14   0.7  77.5   7.1   1.8        0       0

In [19]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [20]:
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
regressor=LinearRegression()
regressor.fit(x_train,y_train)
y_pred=regressor.predict(x_test)
mae=mean_absolute_error(y_true=y_test,y_pred=y_pred)
score=r2_score(y_test,y_pred)

In [22]:
print(mae)
print(score)

0.5468236465249978
0.9847657384266951


Cross Validation

In [23]:
from sklearn.linear_model import LassoCV
Lasso=LassoCV(cv=5)
Lasso.fit(x_train,y_train)
y_pred_Lasso=Lasso.predict(x_test)

In [24]:
mae_lasso=mean_absolute_error(y_test,y_pred_Lasso)
score_Lasso=r2_score(y_test,y_pred_Lasso)
print(mae_lasso)
print(score_Lasso)

0.6358894638564663
0.9814406737205228


In [28]:
from sklearn.linear_model import Ridge
Ridgeregressor=Ridge()
Ridgeregressor.fit(x_train,y_train)
y_pred_Ridge=Ridgeregressor.predict(x_test)

In [29]:
mae_lasso=mean_absolute_error(y_test,y_pred_Ridge)
score_Lasso=r2_score(y_test,y_pred_Ridge)
print(mae_lasso)
print(score_Lasso)

0.5642305340105715
0.9842993364555512


In [30]:
import pickle


In [31]:
scaler

StandardScaler()

In [32]:
Ridgeregressor

Ridge()

In [33]:
pickle.dump(scaler,open("scaler.pkl","wb"))
pickle.dump(Ridgeregressor,open("ridge.pkl","wb"))